# Selenium 웹 자동화 통합 튜토리얼

이 노트북은 Selenium을 사용한 웹 브라우저 자동화에 대한 종합 가이드입니다.

## 목차
1. [Selenium 설치 및 기본 설정](#설치-및-기본-설정)
2. [기본적인 웹 자동화](#기본적인-웹-자동화)
3. [웹 스크래핑과 BeautifulSoup 연동](#웹-스크래핑과-beautifulsoup-연동)
4. [고급 기능: 스크린샷, 페이지 네비게이션](#고급-기능)
5. [모범 사례 및 팁](#모범-사례)

## 설치 및 기본 설정

먼저 필요한 라이브러리들을 import하고 Chrome 드라이버를 설정합니다.

In [2]:
# 필요한 라이브러리 설치
# !conda install selenium beautifulsoup4
# 또는
# !pip install selenium beautifulsoup4

In [1]:
# 필요한 라이브러리 import
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
from bs4 import BeautifulSoup

In [3]:
def setup_chrome_driver(headless=False, window_size=(1920, 1000), detach=True):
    """
    Chrome WebDriver 설정 함수
    
    Parameters:
    - headless: 브라우저 창을 숨길지 여부 (True/False)
    - window_size: 브라우저 창 크기 (width, height)
    - detach: 스크립트 종료 후 브라우저를 유지할지 여부
    
    Returns:
    - driver: 설정된 WebDriver 객체
    """
    chrome_options = Options()
    
    if window_size:
        chrome_options.add_argument(f"--window-size={window_size[0]},{window_size[1]}")
    
    chrome_options.headless = headless
    
    if detach:
        chrome_options.add_experimental_option("detach", True)
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(3)
    
    return driver

## 1. 기본적인 웹 자동화

Google 검색을 자동화하는 예제입니다.

In [5]:
def google_search_example(search_term="파이썬"):
    """
    Google에서 자동 검색을 수행하는 함수
    """
    # WebDriver 설정
    driver = setup_chrome_driver()
    
    try:
        # Google 페이지 열기
        driver.get("http://www.google.com")
        
        # 검색창 찾기 (name 속성이 "q")
        input_box = driver.find_element(By.NAME, "q")
        
        time.sleep(2)
        
        # 검색어 입력
        input_box.send_keys(search_term)
        time.sleep(3)
        
        # 검색 실행 (Enter 키 또는 submit)
        input_box.submit()
        
        print(f"'{search_term}' 검색이 완료되었습니다.")
        print(f"현재 URL: {driver.current_url}")
        
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
    
    return driver

# 실행 예제
driver = google_search_example("파이썬")

'파이썬' 검색이 완료되었습니다.
현재 URL: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3D%25ED%258C%258C%25EC%259D%25B4%25EC%258D%25AC%26sca_esv%3D656b10c93ec889a4%26source%3Dhp%26ei%3D1i2daKiIKaSb2roPkeajoQU%26iflsig%3DAOw8s4IAAAAAaJ075r0InJacEGjBI77HEyNY04vczq6L%26sei%3D3C2daP6NM6Gn2roPlsWVsAI&q=EgQ6eKd-GN3b9MQGIjA5hAQEJKF48nEcOyW6mt3IbkVeeU-ebbGsWkqM2-hVfp2aRlKpH4bkLjiQTGfdXZoyAVJaAUM


## 2. 웹 스크래핑과 BeautifulSoup 연동

Python.org에서 최근 이벤트 정보를 추출하는 예제입니다.

In [6]:
def python_org_scraping():
    """
    Python.org에서 최근 이벤트 정보를 스크래핑하는 함수
    """
    driver = setup_chrome_driver()
    
    try:
        # Python.org 페이지 열기
        driver.get("http://www.python.org")
        
        # 페이지 제목 확인
        assert "Python" in driver.title
        print(f"페이지 제목: {driver.title}")
        
        # 검색창 사용 (선택사항)
        try:
            input_box = driver.find_element(By.NAME, "q")
            time.sleep(2)
            input_box.send_keys("python")
            time.sleep(3)
            input_box.send_keys(Keys.RETURN)
        except:
            print("검색창을 찾을 수 없습니다.")
        
        # 페이지 소스 가져오기
        doc = driver.page_source
        soup = BeautifulSoup(doc, "html.parser")
        
        # 최근 이벤트 정보 추출
        ul = soup.find("ul", {"class": "list-recent-events menu"})
        
        if ul is None:
            print("최근 이벤트를 찾을 수 없습니다.")
            # 다른 클래스명이나 구조로 시도
            events = soup.find_all("li", class_=lambda x: x and "event" in x)
            if events:
                print("다른 방식으로 찾은 이벤트들:")
                for event in events[:5]:  # 최대 5개만
                    print(f"- {event.text.strip()}")
            else:
                print("이벤트 정보를 찾을 수 없습니다.")
        else:
            print("최근 이벤트 목록:")
            li_list = ul.find_all("li")
            for item in li_list:
                print(f"- {item.text.strip()}")
    
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
    
    return driver

# 실행 예제
driver = python_org_scraping()

페이지 제목: Welcome to Python.org
최근 이벤트 목록:
- No results found.


## 3. 고급 기능: 스크린샷, 페이지 네비게이션

버튼 클릭, 스크린샷 저장, CSS 선택자 사용 등의 고급 기능을 다룹니다.

In [7]:
def advanced_selenium_features():
    """
    Selenium의 고급 기능들을 시연하는 함수
    """
    driver = setup_chrome_driver()
    
    try:
        # Python.org 페이지 열기
        driver.get("http://www.python.org")
        
        # 페이지 정보 출력
        assert "Python" in driver.title
        print(f"현재 URL: {driver.current_url}")
        print(f"페이지 제목: {driver.title}")
        
        # 검색 수행
        try:
            input_box = driver.find_element(By.NAME, "q")
            time.sleep(2)
            input_box.send_keys("python")
            time.sleep(3)
            
            # 버튼 클릭 방식으로 검색 실행
            try:
                button = driver.find_element(By.NAME, "submit")
                button.click()
                print("검색 버튼 클릭 완료")
            except:
                # 버튼을 찾을 수 없으면 Enter 키 사용
                input_box.send_keys(Keys.RETURN)
                print("Enter 키로 검색 완료")
        except:
            print("검색 기능을 사용할 수 없습니다.")
        
        # 윈도우 크기 조정 및 스크린샷
        driver.set_window_size(400, 400)
        screenshot_path = "python_event_small.png"
        driver.save_screenshot(screenshot_path)
        print(f"스크린샷 저장: {screenshot_path}")
        
        # 윈도우 크기 재조정
        driver.set_window_size(800, 600)
        
        # CSS 선택자를 사용한 요소 찾기 및 클릭
        print("페이지 네비게이션 시도...")
        try:
            # 다양한 CSS 선택자 시도
            selectors = [
                "#content > div > section > form > div > a:nth-child(1)",
                "#content > div > section > form > div > a",
                "a[href*='next']",
                ".next-page",
                "a:contains('Next')"
            ]
            
            clicked = False
            for selector in selectors:
                try:
                    next_element = driver.find_element(By.CSS_SELECTOR, selector)
                    print(f"선택자 '{selector}'로 요소 발견: {next_element.tag_name}")
                    next_element.click()
                    clicked = True
                    break
                except:
                    continue
            
            if not clicked:
                print("네비게이션 요소를 찾을 수 없습니다.")
            else:
                time.sleep(5)
                print(f"페이지 이동 완료: {driver.current_url}")
        
        except Exception as nav_error:
            print(f"네비게이션 오류: {nav_error}")
    
    except Exception as e:
        print(f"전체 오류: {e}")
    
    finally:
        # 최종 스크린샷
        final_screenshot = "python_final.png"
        try:
            driver.save_screenshot(final_screenshot)
            print(f"최종 스크린샷 저장: {final_screenshot}")
        except:
            print("최종 스크린샷 저장 실패")
    
    return driver

# 실행 예제
driver = advanced_selenium_features()
driver.quit()  # 브라우저 종료

현재 URL: https://www.python.org/
페이지 제목: Welcome to Python.org
검색 버튼 클릭 완료
스크린샷 저장: python_event_small.png
페이지 네비게이션 시도...
선택자 '#content > div > section > form > div > a:nth-child(1)'로 요소 발견: a
페이지 이동 완료: https://www.python.org/search/?q=python&page=2
최종 스크린샷 저장: python_final.png


## 4. 통합 실행 함수

모든 기능을 순차적으로 실행하는 통합 함수입니다.

In [8]:
def run_selenium_tutorial():
    """
    Selenium 튜토리얼의 모든 기능을 순차적으로 실행하는 함수
    """
    print("=" * 50)
    print("Selenium 웹 자동화 튜토리얼 시작")
    print("=" * 50)
    
    drivers = []
    
    try:
        # 1. Google 검색 예제
        print("\n1. Google 검색 자동화...")
        driver1 = google_search_example("파이썬 selenium")
        drivers.append(driver1)
        
        time.sleep(3)
        
        # 2. Python.org 스크래핑
        print("\n2. Python.org 웹 스크래핑...")
        driver2 = python_org_scraping()
        drivers.append(driver2)
        
        time.sleep(3)
        
        # 3. 고급 기능 시연
        print("\n3. 고급 기능 시연...")
        driver3 = advanced_selenium_features()
        drivers.append(driver3)
        
        print("\n=" * 50)
        print("모든 튜토리얼이 완료되었습니다!")
        print("생성된 파일들:")
        print("- python_event_small.png")
        print("- python_final.png")
        print("=" * 50)
        
    except Exception as e:
        print(f"튜토리얼 실행 중 오류: {e}")
    
    # 사용자에게 브라우저 종료 여부 확인
    response = input("\n모든 브라우저를 종료하시겠습니까? (y/n): ")
    if response.lower() == 'y':
        for driver in drivers:
            try:
                driver.quit()
            except:
                pass
        print("모든 브라우저가 종료되었습니다.")
    else:
        print("브라우저들이 열린 상태로 유지됩니다.")
    
    return drivers

# 전체 튜토리얼 실행
drivers = run_selenium_tutorial()

Selenium 웹 자동화 튜토리얼 시작

1. Google 검색 자동화...
'파이썬 selenium' 검색이 완료되었습니다.
현재 URL: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3D%25ED%258C%258C%25EC%259D%25B4%25EC%258D%25AC%2Bselenium%26sca_esv%3D656b10c93ec889a4%26source%3Dhp%26ei%3DoDCdaJfQB76N2roPjeCDkQ8%26iflsig%3DAOw8s4IAAAAAaJ0-sF2YqHCJhvzVa1U_vJlv7Xqx1X_V%26sei%3DpjCdaKmKBI7G0-kP9dKU2Q8&q=EgQ6eKd-GKbh9MQGIjBTXmHl3fd-l01cBp0vcAAuVeg_1Q5ovdwsSMBKodiOmbWkOl8m0glvC-vd7V8yNLsyAVJaAUM

2. Python.org 웹 스크래핑...
페이지 제목: Welcome to Python.org
최근 이벤트 목록:
- Download Python for other platforms

    ...Python binary.


Python for AIX
AIX binary packages for Python are available from IBM AIX Toolbox in RPM format. They can be installed using dnf package manager. Visit the Get Started page for more details.




Python for HP-UX
You can purchase ActivePython
(commercial and community versions, including scientific computing modules, not open source)


Python for IBM i (formerly AS/400, iSeries)
Both Python 2 and 

## 5. 모범 사례 및 팁

### 주요 학습 포인트:

1. **WebDriver 설정**:
   - `Options` 객체로 브라우저 설정 관리
   - `headless` 모드로 백그라운드 실행 가능
   - `detach` 옵션으로 스크립트 종료 후에도 브라우저 유지

2. **요소 찾기 방법**:
   - `By.NAME`: HTML name 속성으로 찾기
   - `By.CSS_SELECTOR`: CSS 선택자로 찾기
   - `By.XPATH`: XPath로 찾기

3. **상호작용 방법**:
   - `send_keys()`: 텍스트 입력
   - `click()`: 클릭 이벤트
   - `submit()`: 폼 제출
   - `Keys.RETURN`: 특수 키 입력

4. **데이터 추출**:
   - `page_source`: HTML 소스 코드 가져오기
   - BeautifulSoup와 연동하여 파싱
   - CSS 선택자와 정규표현식 활용

5. **유틸리티 기능**:
   - `save_screenshot()`: 화면 캡처
   - `set_window_size()`: 창 크기 조정
   - `current_url`, `title`: 페이지 정보 확인

### 주의사항:
- 항상 `time.sleep()`으로 적절한 대기 시간 설정
- `try-except` 블록으로 예외 처리
- 사용 후 `driver.quit()`으로 리소스 정리
- 웹사이트의 robots.txt와 이용약관 준수

## 실습 과제

다음 과제들을 직접 구현해보세요:

1. **뉴스 사이트 스크래핑**: 특정 뉴스 사이트에서 최신 기사 제목들을 추출
2. **온라인 쇼핑몰 가격 비교**: 여러 쇼핑몰에서 같은 제품의 가격을 비교
3. **소셜미디어 자동 포스팅**: (개인 계정에서만) 자동으로 포스트 작성
4. **웹사이트 모니터링**: 특정 웹페이지의 변경사항을 주기적으로 확인

In [ ]:
# 여기에 실습 코드를 작성해보세요!

def custom_automation_task():
    """
    사용자 정의 자동화 작업을 구현하는 함수
    """
    pass

# 실습 코드 실행
custom_automation_task()